# Resume Optimizer using AI
**Shout out to [Shaw Tabeli](https://www.shawhintalebi.com/) for the solid foundation and inspiration**
- Install the Libraries

In [52]:
# Install Libs
!pip install -U -q google-generativeai markdown weasyprint

**Import the Libraries**

In [16]:
from IPython.display import display, Markdown
import os
import google.generativeai as genai
from markdown import markdown
from weasyprint import HTML

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


### Import Resume and Job Description
- Create a markdown version of your resume
- Read the your resume assign it to the resume_string variable

In [ ]:
# Open and Read the resume.md 
with open("resume/morrison_resume.md", "r", encoding="utf-8") as file:
    resume_string = file.read()

display(Markdown(resume_string))

### Input the Job Description
- Copy the job description
- Past it into the box below the prompt

In [ ]:
# Prompt user for job data
jd_string = "" 
while not jd_string:  # Loop until we get some input
    print("To help me analyze this job description for you, please paste the full text below:")
    jd_string = input()
    print(jd_string)
    if not jd_string:
        print("Oops! Looks like you didn't paste anything. Please try again.")

### Build the Prompt Template
- Provide instructions to the LLM for formatting the output of your resume.
- Pass the your resume and job descriptions as variables to the prompt. 

In [28]:
prompt_template = lambda resume_string, jd_string : f"""
You are a professional resume optimization expert specializing in tailoring resumes to specific job descriptions. Your goal is to optimize my resume and provide actionable suggestions for improvement to align with the target role.

### Guidelines:
1. **Relevance**:  
   - Prioritize experiences, skills, and achievements **most relevant to the job description**.  
   - Remove or de-emphasize irrelevant details to ensure a **concise** and **targeted** resume.
   - Limit work experience section to 4-5 most relevant roles
   - Limit bullet points under each role to 4-5 most relevant impacts

2. **Action-Driven Results**:  
   - Use **strong action verbs** and **quantifiable results** (e.g., percentages, revenue, efficiency improvements) to highlight impact.  

3. **Keyword Optimization**:  
   - Integrate **keywords** and phrases from the job description naturally to optimize for ATS (Applicant Tracking Systems).  

4. **Additional Suggestions** *(If Gaps Exist)*:  
   - If the resume does not fully align with the job description, suggest:  
     1. **Additional technical or soft skills** that I could add to make my profile stronger.  
     2. **Certifications or courses** I could pursue to bridge the gap.  
     3. **Project ideas or experiences** that would better align with the role.  

5. **Formatting**:  
   - Output the tailored resume in **clean Markdown format**.  
   - Include an **"Additional Suggestions"** section at the end with actionable improvement recommendations.  
   - Aim for a resume length of around 750-1000 words.
   - Tailor the tone and style to match the target company and industry.

---

### Input:
- **My resume**:  
{resume_string}

- **The job description**:  
{jd_string}

---

### Output:  
 **Tailored Resume**:  
   - A resume in **Markdown format** that emphasizes relevant experience, skills, and achievements.  
   - Incorporates job description **keywords** to optimize for ATS.  
   - Uses strong language and is no longer than **two pages**.

 **Contact Information:**  <-- Separate section for contact info
    - Include all provided contact details (email, phone, LinkedIn, GitHub) in a list format.

 **Certifications:**  <-- Separate section for certifications at the bottom
    - Include all provided certifications with links.

 **Additional Suggestions** *(if applicable)*:  
     # Suggestions for Improvement
   - Determine if the resume and experience is a **FIT** and a good candidate for this specific role.
   - List **skills** that could strengthen alignment with the role.  
   - Recommend **certifications or courses** to pursue.  
   - Suggest **specific projects or experiences** to develop.
"""

**List the available models availble from Google**

In [ ]:
# List available models
for m in genai.list_models():
    print(f"Model: {m.name}")

**Call the `prompt_template` function and assign it to a variable for passing to the model**

In [30]:
prompt = prompt_template(resume_string, jd_string)

### Text Generation with Gemini

**1. Model Configuration:**

Define a dictionary `generation_config` to customize the text generation process. This includes parameters like:

*   **temperature:** Controls the randomness of the output. Lower values make the output more focused and deterministic.
*   **top_p:**  Uses nucleus sampling to limit token selection to the top p probability mass.
*   **top_k:** Limits token selection to the top k most likely tokens.
*   **max_output_tokens:** Sets the maximum number of tokens in the generated response.
*   **response_mime_type:** Specifies the format of the generated response (e.g., "text/plain").

**2. Model Initialization:**

Create a `GenerativeModel` object, providing the model name ("gemini-1.5-pro") and the `generation_config`.

**3. Generating Text:**

- The `generate_content` method sends a `prompt` to the model, triggering the text generation process.
- The generated text is then extracted from the `response` object and printed to the console.

In [ ]:
# Create the model
generation_config = {
  "temperature": .5,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  generation_config=generation_config,
)


# Query the model
response = model.generate_content(prompt) 
generated_text = response.text
print(generated_text)

Split out the updated text for the resume and suggestions

In [32]:
resume_list = generated_text.split("# Suggestions for Improvement")

Display the optimized resume in markdown

In [ ]:
display(Markdown(resume_list[0]))

### Save File to PDF & HTML

In [ ]:
# Save to PDF
output_resume_pdf = "resume/gemini_resume_new.pdf"

# Convert Markdown to HTML
html_output = markdown(resume_list[0])
print(html_output)

In [ ]:
from markdown import markdown

def markdown_to_html_with_css(markdown_text, css_file="styles.css", google_font="Roboto"):
  """
  Converts markdown text to HTML, applies an external CSS file, and adds a Google Font.

  Args:
    markdown_text: The markdown text to convert.
    css_file: The path to the CSS file.
    google_font: The name of the Google Font to use.

  Returns:
    The HTML output as a string.
  """

  html_output = markdown(markdown_text)

  # Construct the HTML with the linked CSS file and Google Font
  html_with_css = f"""
  <!DOCTYPE html>
  <html>
  <head>
    <meta charset="UTF-8">
    <title>Resume</title>
    <link rel="stylesheet" href="{css_file}">
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family={google_font}&display=swap" rel="stylesheet">
  </head>
  <body>
    {html_output}
  </body>
  </html>
  """

  return html_with_css

# Example usage with resume_list and a Google Font:
html_output = markdown_to_html_with_css(resume_list[0], google_font="Open+Sans")

# Now you can save html_output to a file or use it further
print(html_output)

# Specify the filename
filename = "resume/new_gemini_resume.html"

# Write the HTML to the file
with open(filename, "w") as f:
  f.write(html_output)

print(f"HTML file saved as {filename}")

Write the file to pdf

In [37]:
# Save to PDF
output_resume_pdf = "resume/gemini_resume_new.pdf"

# Convert HTML to PDF and save
HTML(string=html_output).write_pdf(output_resume_pdf, stylesheets=['resume/styles.css'])

Write the updated resume to markdown

In [39]:
# save as markdown
output_file = "resume/gemini_resume_new.md"

with open(output_file, "w", encoding="utf-8") as file:
    file.write(resume_list[0])

Display the suggestions and if role fit

In [ ]:
display(Markdown(resume_list[1]))